In [27]:
# coding=utf-8
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

# 确保可重现，指定随机数
seed = 13
np.random.seed(seed)
tf.set_random_seed(seed)

In [28]:
num_channels = 4
history_length = 20
batch_size = 1
split_size = 2 
window_size =10 

In [36]:
input_shape = (history_length, num_channels)
state = np.zeros(input_shape)
state = np.random.randint(1,5,size=input_shape)
state

array([[3, 3, 1, 3],
       [1, 2, 3, 1],
       [3, 1, 3, 4],
       [4, 4, 1, 4],
       [1, 3, 3, 1],
       [1, 3, 1, 2],
       [1, 3, 2, 2],
       [1, 1, 2, 3],
       [3, 1, 3, 2],
       [4, 2, 3, 4],
       [3, 4, 1, 4],
       [3, 1, 2, 1],
       [3, 4, 2, 2],
       [3, 4, 1, 3],
       [1, 1, 3, 3],
       [4, 1, 3, 3],
       [2, 2, 4, 1],
       [3, 1, 4, 2],
       [1, 3, 1, 3],
       [4, 4, 4, 1]])

In [19]:
s_t = tf.placeholder(dtype=tf.float32, shape=[None, history_length, num_channels])
# input_3d = tf.expand_dims(ohlc, 0)  # shape (30,4) --> (1,30,4)
reshape = tf.reshape(s_t, shape=[batch_size, split_size, window_size, num_channels])

In [5]:
kernel_size = 2
filter_size = 3 # 滤波器数量，1输出1列数据，n输出n列数据
conv2d = tf.layers.conv2d(inputs=reshape, filters=filter_size,
                          kernel_size=[1, kernel_size], strides=(1, 1),
                          padding='valid', activation=None)
'''列卷积'''
normal = tf.layers.batch_normalization(inputs=conv2d, training=True, scale=True)
'''对一列数据做正规化，独立计算列，如果一列数据都相同，此列输出0'''

input_shape = tf.shape(normal)
window_size = window_size - kernel_size + 1
reshape1 = tf.reshape(normal, shape=[batch_size, split_size, window_size * filter_size])

In [16]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    result = sess.run(fetches=s_t, feed_dict={s_t: [state]})
    print(result)
    result = sess.run(fetches=conv2d, feed_dict={s_t: [state]})
    print(result)
    result = sess.run(fetches=normal, feed_dict={s_t: [state]})
    print(result)
    result = sess.run(fetches=reshape1, feed_dict={s_t: [state]})
    print(result)

[[[ 3.  1.  3.  1.]
  [ 3.  3.  1.  2.]
  [ 1.  3.  4.  3.]
  [ 1.  3.  3.  3.]
  [ 3.  2.  2.  3.]
  [ 2.  4.  1.  3.]
  [ 1.  4.  2.  4.]
  [ 4.  3.  2.  4.]
  [ 2.  3.  1.  1.]
  [ 4.  1.  1.  3.]
  [ 3.  3.  1.  2.]
  [ 4.  4.  2.  1.]
  [ 4.  3.  1.  2.]
  [ 2.  1.  4.  2.]
  [ 2.  4.  4.  2.]
  [ 3.  4.  3.  3.]
  [ 4.  4.  4.  4.]
  [ 3.  3.  4.  1.]
  [ 4.  3.  1.  3.]
  [ 2.  3.  2.  3.]]]
[[[[-1.33873022  0.8012287  -1.79627013]
   [-1.75139332  0.18181691 -3.24360108]
   [-0.18664128  1.7544018  -2.17031693]
   [-1.01400137  0.27651498 -3.02645421]
   [-1.00606132  2.18393707 -2.60098767]
   [-1.9602375   1.32021248 -2.85595274]
   [-1.78900886 -0.24253905 -3.78517866]
   [-1.16652465  3.89439344 -3.80495882]
   [-3.05185342 -1.77843368 -3.40964746]]

  [[-2.74427557  0.26360187 -2.51724672]
   [-4.0366416   0.50869912 -3.35714817]
   [-2.08032942 -0.12029248 -4.72506046]
   [ 0.14745891  0.817056   -1.36440849]
   [-2.25442433  0.63927352 -2.40009427]
   [-2.46265817 -0.313

In [7]:
gru_cells = []
train = True
gru_keep_prob = 0.5
gru_num_cells = 2
gru_cell_size = 128
for i in range(0, gru_num_cells):
    cell = tf.contrib.rnn.GRUCell(num_units=gru_cell_size)                
    if train:
        cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=gru_keep_prob)
        gru_cells.append(cell)
        
multicell = tf.contrib.rnn.MultiRNNCell(gru_cells)
output, final_state = tf.nn.dynamic_rnn(cell=multicell, inputs=reshape1, dtype=tf.float32)
out = tf.unstack(output, axis=1)[-1]

In [8]:
with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    result = sess.run(fetches=out, feed_dict={s_t: [state]})
    print(result)

[[-0.         -0.00698679 -0.          0.06069346  0.          0.08136414
  -0.         -0.          0.          0.03657795  0.         -0.02237264
   0.         -0.0666007   0.         -0.         -0.04022884 -0.05512542
  -0.17065185 -0.02072918 -0.          0.17627537 -0.         -0.
   0.02873364 -0.08650163  0.055222    0.         -0.04822397  0.16433915
  -0.16346392  0.          0.10936277  0.0855329  -0.02512092  0.
   0.03830057 -0.13932312 -0.          0.1262095   0.20142992  0.         -0.
   0.         -0.01664362  0.00919864 -0.         -0.         -0.04053437
  -0.13862249  0.09863307  0.         -0.19957796  0.          0.16707429
  -0.07691921  0.         -0.10159691  0.          0.         -0.01239831
  -0.         -0.          0.          0.          0.          0.02715179
  -0.03571545 -0.         -0.          0.03334166  0.07660545 -0.         -0.
  -0.04066686  0.          0.02586804  0.         -0.01981907  0.
  -0.01123836  0.         -0.          0.0801781  -0.0

In [11]:
output

<tf.Tensor 'rnn/transpose_1:0' shape=(1, 2, 128) dtype=float32>